# Predicting heart disease using machine learning

This notebook looks into using various Python-based machine learning and data science libraries in a attempt to build a machine learning capable of predicting whether or not someone has heart disease based on their medical attributes.